In [2]:
import pandas as pd
from viral_utils import ActivityPipelineRunner
from data_proc_utils import denoise_VAL
from ww_analyzer import PermutationEntropyAnalyzer, GetOptimalWindow, GrangerCausalityAnalyzer
from viral_utils import ViralActivityProcessor


In [3]:
raw_data_path = 'raw_data/wastewater_viral_load_virginia.csv'
sewershed_list_path = 'misc_data/sewersheds_virginia.txt'
sewershed_output_dir = './processed_data/viral_activity_level/sewershed/'
merged_output_dir = './processed_data/viral_activity_level/'



### Raw data

In [4]:
raw_data=pd.read_csv('./raw_data/wastewater_viral_load_virginia.csv')

In [10]:
raw_data.head()

,geo_value,EW_stdate,log_viral_load,region,geo_res
0,Alexandria Renew,2021-09-12,28.631653,Northern,sewershed
1,Alexandria Renew,2021-09-19,27.862133,Northern,sewershed
2,Alexandria Renew,2021-09-26,28.746163,Northern,sewershed
3,Alexandria Renew,2021-10-03,27.342005,Northern,sewershed
4,Alexandria Renew,2021-10-10,28.455197,Northern,sewershed


In [9]:
raw_data.geo_value.unique()

array(['Alexandria Renew', 'Aquia', 'Arlington', 'Army Base', 'Atlantic',
       'Blacksburg', 'Boat Harbor', 'Broad Run', 'Clintwood',
       'Coeburn Norton Wise', 'Farmville', 'HL Mooney', 'Halifax',
       'Henrico', 'Hillsville', 'James River', 'Lebanon',
       'Little Falls Run', 'Low Moor', 'Lower Jackson', 'Lynchburg',
       'Moores Creek', 'Nansemond', 'North River', 'Onancock',
       'Parkins Mill', "Pepper's Ferry", 'Pound', 'Richmond', 'Roanoke',
       'Rocky Mount', 'Tazewell', 'Upper Occoquan',
       'Virginia Initiative Plant', 'Williamsburg', 'Wolf Creek',
       'York River'], dtype=object)

### Convert Viral load to VAL, merge sewershed level data to region and state level using ActivityPipelineRunner

In [ ]:
pipeline = ActivityPipelineRunner(
    raw_data_path=raw_data_path,
    sewershed_list_path=sewershed_list_path,
    output_dir=sewershed_output_dir,
    merged_output_dir=merged_output_dir
)
pipeline.run()

In [ ]:
mergedf=pd.DataFrame()

rdf=pd.read_csv('processed_data/viral_activity_level/region_viral_activity_level.csv',parse_dates=['time_value'])
sdf=pd.read_csv('processed_data/viral_activity_level/state_viral_activity_level.csv',parse_dates=['time_value'])

mergedf=pd.concat([rdf,sdf])
mergedf.time_value=mergedf.time_value+pd.Timedelta(days=6)

hdf=pd.read_csv('processed_data/hosp_region/hosp_region_virginia.csv',parse_dates=['time_value'])

mergedf=mergedf[['geo_value','time_value','geo_res','viral_activity_level']].merge(hdf,how='outer')

mergedf.to_csv('processed_data/alldata_combined.csv',index=None)